In [1]:
# !pip freeze > requirements.txt

In [2]:
# !pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np
import json
import os
from collections import defaultdict, deque
import pickle
from IPython import get_ipython
from tqdm.auto import tqdm
import gc
import subprocess
from IPython.display import clear_output
import matplotlib.pyplot as plt
import wandb

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cuda


In [2]:
os.getcwd()

'/media/michael/Fichiers/Fac/Thèse 1/Code/Scripts'

# Metrics

In [ ]:
os.chdir('./Reward_weights')

metrics = ['v_degraded', 
           'v1_not_sent_from_s1', 
           'v3_not_sent_from_s3', 
           'v_not_found_in_last_station', 
           'z1_VSAV_sent', 
           'rupture_ff']

for m in metrics:

    dic_tarif_sent_disp = {'v_required': 0,
                    'v_sent': 0,
                    'v_sent_full':0,
                    'v_degraded':0,
                    'cancelled':0, #cancel departure
                    'function_not_found':0,
                    'v1_not_sent_from_s1':0,
                    'v3_not_sent_from_s3':0,
                    'v_not_found_in_last_station':0,
                    'ff_required':0,
                    'ff_sent':0,
                    'rupture_ff':0,       
                    'z1_VSAV_sent': 0,
                    'z1_FPT_sent': 0,
                    'z1_EPA_sent': 0,
                     'VSAV_needed':0,
                     'FPT_needed':0,
                     'EPA_needed':0,
                     'VSAV_disp':0,
                     'FPT_disp':0,
                     'EPA_disp':0,
                    'skill_lvl':0
                    } 

    dic_tarif_sent_disp[m] = -100

    if m == 'v_degraded':
        
        dic_tarif_sent_disp['v_sent_full'] = 10


    with open(f"rw_"+ m +".json", "w") as f:
        json.dump(dic_tarif_sent_disp, f)

os.chdir("../")

In [ ]:
os.chdir("../")

In [ ]:
os.getcwd()

# Simulation

In [ ]:
cmd = [
    "python3", "-u", "simulation_start.py",
    "--dataset", "df_pc_real_prob.pkl",
    "--start", "1",
    "--end", "53088",
    "--constraint_factor_ff", "1",
    "--constraint_factor_veh", "1",
    "--save_metrics_as", f"sim_rdm_prob"
]

#     "--is_best",
# 636960

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

In [ ]:
# TO DEBUG

# print(''.join(stdout_lines[-150:]))

In [ ]:
# 53000 v_out: 20 rwd_mean: -1.00 score: -209800 act: 255849 ratio:, 0.820 v_not_found_ls: 185 cancel_dep: 31628

# Agent

## Params

In [ ]:
action_size = 80
train_seed = 41

os.chdir("./Data")


hyper_params = {"state_size" : (action_size + 2) *40,
                "action_size" : action_size,
                "layer_type" : "ff", # noisy else ff
                "layer_size" : 1024,
                "num_layers" : 8,
                "use_batchnorm" : True,
                "n_steps" : 20, #5, 1, 50, 64
                "batch_size" : 256, #512, 256, 64, 32, 16, 8
                "buffer_size" : 100000, #100000, 128, 5000, 10000, 65 buffer_size > batch_size
                "update_every" : 32, #1, 200, 500, 64, 32 Q updates
                "per" : 1, # 0 for curiosity > 0, else 1, 2
                "rdm" : 0, # only if not per
                "munchausen" : 1, #1
                "curiosity" : 0, #Adds intrinsic curiosity to the extrinsic reward. 0 - only reward/ no curiosity, 
                                                                                    #1 - reward and curiosity, 
                                                                                    #2 - only curiosity
                "curiosity_size" : 1024,
                "lr" : 1e-4, #1e-3, 5e-4, 5e-3, 1e-4
                "lr_dec" : 1, #0, 1, 2, 3
                "entropy_tau" : 0.03, #0.03, 0.05 idem #-  Munch param
                "entropy_tau_coeff" : 1e-2, #1e-2 #-  Munch param
                "lo" : -1, #-  Munch param
                "alpha" : 0.9, #-  Munch param
                "gamma" : 0.99,
                "tau" : 0.005, #1e-2, 5e-3
                "N" : 32,# Number of quantiles 32, 64
                "entropy_coeff" : 0.001,
                "decay_update" : 100,
                "device" : str(device),
                "seed" : train_seed}


json.dump(hyper_params, open("hyper_params.json", "w"))

os.chdir("../")

## Popen

In [ ]:
os.getcwd()

In [ ]:
os.chdir("../")

In [ ]:
model = "fqf"
years = "10y"
suffix = "cfv1p1"
rwd="rupture_ff"

cmd = [
    "python3", "-u", "agent_run.py",
    "--model_name", f"agent_{model}_{years}_{suffix}_{rwd}",
    "--agent_model", model,
    "--hyper_params", "hyper_params.json",
    "--reward_weights", f"rw_{rwd}.json",
    "--dataset", f"df_pc_fake_{years}.pkl",
    "--start", "1",
    "--end", "530880",
    "--eps_start", "1",
    "--constraint_factor_veh", "1",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"metrics_{model}_{years}_{suffix}_{rwd}",
    "--train"

]
# "--load"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

In [ ]:
64*82

In [ ]:
# TO DEBUG

print(''.join(list(stdout_lines)[-1:]))

In [ ]:
# 53000 v_out: 11 rwd_mean: -2.00 score: -222700 act: 255395 ratio:, 0.87198 v_not_found_ls: 175 cancel_dep: 30864 eps:, 0.10, lr:, 0.00077
# 159200 v_out: 10 rwd_mean: 0.00 score: -425000 act: 796780 ratio: 0.53340 v_not_found_ls: 786 cancel: 120630 degraded: 9444 eps: 0.00 lr: 0.00029

In [ ]:
model = "fqf"
years = "10y"
suffix = "nsteps_20"

cmd = [
    "python3", "-u", "agent_run.py",
    "--model_name", f"agent_{model}_{years}_{suffix}",
    "--agent_model", model,
    "--hyper_params", "hyper_params.json",
    "--reward_weights", f"rw_sent_disp.json",
    "--dataset", f"df_pc_real.pkl",
    "--start", "1",
    "--end", "53088",
    "--eps_start", "0",
    "--constraint_factor_veh", "3",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"agent_metrics_{model}_{years}_{suffix}_test",
]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = []
for line in process.stdout:
    # stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

In [ ]:
#z1 sent inter 10000 : 757

# POMO

In [ ]:
action_size = 80
feature_size = 40
train_seed = 41

os.chdir("./Data")


hyper_params = {"state_size" : (action_size + 2) * feature_size, 
                "action_size": action_size,
                "feature_size": feature_size,
                "batch_size": 256,
                "update_every":32,
                "layer_size" : 1024,
                "num_layers" : 8,
                "use_batchnorm" : True,
                "d_model": 64, 
                "n_heads": 4,  
                "lr": 1e-4, 
                "num_samples": 8, 
                "buffer_size": 10000,
                "device" : str(device),
                "seed" : train_seed}


json.dump(hyper_params, open("hyper_params_pomo.json", "w"))

os.chdir("../")

In [10]:
model = "pomo"
years = "10y"
suffix = "cfv1p1"
rwd="rupture_ff"

cmd = [
    "python3", "-u", "POMO_run.py",
    "--model_name", f"agent_{model}_{years}_{suffix}_{rwd}",
    "--agent_model", model,
    "--hyper_params", "hyper_params_pomo.json",
    "--reward_weights", f"rw_{rwd}.json",
    "--dataset", f"df_pc_fake_{years}.pkl",
    "--start", "1",
    "--end", "530880",
    "--constraint_factor_veh", "1",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"metrics_{model}_{years}_{suffix}_{rwd}",
    "--train"

]
# "--load"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 128])
3 torch.Size([1, 128]) torch.Size([1, 128]) torch.Size([1, 10240])
2 torch.Size([1, 10496])
1 torch.Size([1, 80, 12

KeyboardInterrupt: 

In [7]:
82*64

5248

# Decision Transformer

In [ ]:
os.getcwd()

In [3]:
action_size = 80
feature_size = 40
train_seed = 41

os.chdir("./Data")


hyper_params = {"state_size" : (action_size + 2) * feature_size, 
                "action_size": action_size,
                "feature_size": feature_size,
                "batch_size": 128,
                "update_every":32,
                "layer_size" : 256,
                "num_layers" : 3,
                "max_len" : 20,
                "lr": 1e-4, 
                "buffer_size": 10000,
                "device" : str(device),
                "seed" : train_seed}


json.dump(hyper_params, open("hyper_params_dt.json", "w"))

os.chdir("../")

In [ ]:
model = "dt"
years = "10y"
suffix = "cfv1p1"
rwd="rupture_ff"

cmd = [
    "python3", "-u", "DT_run.py",
    "--model_name", f"agent_{model}_{years}_{suffix}_{rwd}",
    "--agent_model", model,
    "--hyper_params", "hyper_params_dt.json",
    "--reward_weights", f"rw_{rwd}.json",
    "--dataset", f"df_pc_fake_{years}.pkl",
    "--start", "1",
    "--end", "530880",
    "--constraint_factor_veh", "1",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"metrics_{model}_{years}_{suffix}_{rwd}",
    "--train"

]
# "--load"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

228100 z1_VSAV_sent: 542 | z1_FPT_sent: 0 | z1_EPA_sent: 0 | VSAV_disp: 6 | FPT_disp: 6 | EPA_disp: 0 | loss: 3.7625341064995155e-07
228200 v_out: 12 | rwd_mean: -3.00 | v1notfroms1: 47973 | v3notfroms3: 43 | v_not_found_ls: 605 | deg: 4114 | store: 45048
228200 z1_VSAV_sent: 542 | z1_FPT_sent: 0 | z1_EPA_sent: 0 | VSAV_disp: 8 | FPT_disp: 6 | EPA_disp: 0 | loss: 2.1327215904420882e-07
228300 v_out: 14 | rwd_mean: -1.00 | v1notfroms1: 47980 | v3notfroms3: 43 | v_not_found_ls: 605 | deg: 4114 | store: 45068
228300 z1_VSAV_sent: 542 | z1_FPT_sent: 0 | z1_EPA_sent: 0 | VSAV_disp: 5 | FPT_disp: 6 | EPA_disp: 0 | loss: 2.2748359697288834e-05
228400 v_out: 6 | rwd_mean: -1.00 | v1notfroms1: 47988 | v3notfroms3: 43 | v_not_found_ls: 605 | deg: 4114 | store: 45083
228400 z1_VSAV_sent: 542 | z1_FPT_sent: 0 | z1_EPA_sent: 0 | VSAV_disp: 8 | FPT_disp: 6 | EPA_disp: 0 | loss: 5.995629180688411e-06
228500 v_out: 11 | rwd_mean: -5.00 | v1notfroms1: 48009 | v3notfroms3: 43 | v_not_found_ls: 606 | deg

In [5]:
model = "dt"
years = "10y"
suffix = "cfv1p1"
rwd="rupture_ff"

cmd = [
    "python3", "-u", "DT_run.py",
    "--model_name", f"agent_{model}_{years}_{suffix}_{rwd}",
    "--agent_model", model,
    "--hyper_params", "hyper_params_dt.json",
    "--reward_weights", f"rw_{rwd}.json",
    "--dataset", f"df_pc_real.pkl",
    "--start", "1",
    "--end", "53088",
    "--constraint_factor_veh", "1",
    "--constraint_factor_ff", "1",
    "--save_metrics_as", f"metrics_{model}_1y_{suffix}_{rwd}"

]
# "--load"

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

line_count = 0
stdout_lines = deque(maxlen=5000)
for line in process.stdout:
    stdout_lines.append(line)
    line_count += 1
    if line_count % 100 == 0:
        clear_output(wait=True)

    print(line.strip())

Agent dt initialized
Eval mode - weights loaded
Reward weights {'v_required': 0, 'v_sent': 0, 'v_sent_full': 0, 'v_degraded': 0, 'cancelled': 0, 'function_not_found': 0, 'v1_not_sent_from_s1': 0, 'v3_not_sent_from_s3': 0, 'v_not_found_in_last_station': 0, 'ff_required': 0, 'ff_sent': 0, 'rupture_ff': -100, 'z1_VSAV_sent': 0, 'z1_FPT_sent': 0, 'z1_EPA_sent': 0, 'VSAV_needed': 0, 'FPT_needed': 0, 'EPA_needed': 0, 'VSAV_disp': 0, 'FPT_disp': 0, 'EPA_disp': 0, 'skill_lvl': 0}
constraint factor veh is  1
constraint factor ff is  1 Number of ff: 3343


KeyboardInterrupt: 

In [17]:
os.chdir("../")

# Stats

In [ ]:
# 53000 v_out: 21 rwd_mean: 0.00 score: -210800 act: 256561 ratio:, 0.82164 v_not_found_ls: 187 cancel_dep: 30515 eps:, 0.00

In [ ]:
 os.chdir('../')

In [7]:
folder_path = "./Plots"

data = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith(".pkl"):
        full_path = os.path.join(folder_path, file_name)
        with open(full_path, "rb") as f:
            d = pickle.load(f)
            name = os.path.splitext(file_name)[0]
            data[name] = d

df = pd.DataFrame.from_dict(data, orient='index')

df["skill_lvl_mean"] = df["skill_lvl"] / df["ff_sent"]
# To maximize:

df = df.rename(columns={"v_sent": "v_sent ↑", "v_sent_full": "v_sent_full ↑", "ff_sent":"ff_sent ↑"})

# To minimize:

df = df.rename(columns={"v_degraded": "v_degraded ↓", 
                        "z1_VSAV_sent": "z1_VSAV_sent ↓", 
                        "z1_FPT_sent": "z1_FPT_sent ↓", 
                        "z1_EPA_sent": "z1_EPA_sent ↓", 
                        "v1_not_sent_from_s1": "v1_not_sent_from_s1 ↓", 
                        "v3_not_sent_from_s3": "v3_not_sent_from_s3 ↓", 
                        "v_not_found_in_last_station": "v_not_found_in_ls ↓"})


df[["v_required","v_sent ↑","v_sent_full ↑", "v_degraded ↓", "v1_not_sent_from_s1 ↓", "v3_not_sent_from_s3 ↓",
    "v_not_found_in_ls ↓", "rupture_ff", 
    'z1_VSAV_sent ↓', 'z1_FPT_sent ↓', 'z1_EPA_sent ↓',
"ff_sent ↑", "skill_lvl_mean"]]

,v_required,v_sent ↑,v_sent_full ↑,v_degraded ↓,v1_not_sent_from_s1 ↓,v3_not_sent_from_s3 ↓,v_not_found_in_ls ↓,rupture_ff,z1_VSAV_sent ↓,z1_FPT_sent ↓,z1_EPA_sent ↓,ff_sent ↑,skill_lvl_mean
sim_best,62985,63531,62883,648,16331,11,102,10259,127,0,0,198557,1.134390
metrics_dt_10y_cfv1p1_rupture_ff,62985,63616,62901,715,11311,16,84,5253,196,0,0,198260,1.472112
sim_rdm_cf1_cv1_p1,62985,63616,62901,715,11311,16,84,5253,196,0,0,198260,1.474685
sim_rdm_prob,71991,72344,71490,854,24450,82,381,8747,340,0,0,224585,1.491787
sim_rdm_cf1_cv3_p1,62985,63654,62853,801,21154,67,133,6321,2931,288,27,195420,1.548470
sim_rdm_cf1_cv1_p12,75478,76330,75354,976,16139,12,124,7271,371,0,0,237360,1.481543
agent_metrics_fqf_1y_cfv1_p12_rupture_ff,75478,76449,75356,1093,14507,9,122,5385,376,0,0,237453,1.572433
sim_bst_cf1_cv1_p12,75478,76204,75304,900,24387,10,174,15300,241,0,0,237726,1.151990
sim_bst_cf1_cv3_p1,62985,63555,62820,735,27088,99,178,12535,2971,314,26,195315,1.205207
sim_bst_cf3_cv1_p1,62985,61944,51426,10518,220143,580,11559,236488,0,0,0,182683,1.874466


In [12]:
os.chdir("./Plots")

In [13]:
with open("metrics_dt_10y_cfv1p1_rupture_ff.pkl", "rb") as f:
    met = pickle.load(f)

In [14]:
met

{'v_required': 62985,
 'v_sent': 63616,
 'v_sent_full': 62901,
 'v_degraded': 715,
 'rupture_ff': 5253,
 'function_not_found': 6633,
 'v1_not_sent_from_s1': 11311,
 'v3_not_sent_from_s3': 16,
 'v_not_found_in_last_station': 84,
 'ff_required': 0,
 'ff_sent': 198260,
 'z1_VSAV_sent': 196,
 'z1_FPT_sent': 0,
 'z1_EPA_sent': 0,
 'VSAV_needed': 208,
 'FPT_needed': 0,
 'EPA_needed': 0,
 'VSAV_disp': 8,
 'FPT_disp': 5,
 'EPA_disp': 0,
 'skill_lvl': 291861.0}

In [15]:
with open("sim_rdm_cf1_cv1_p1.pkl", "rb") as f:
    met2 = pickle.load(f)
met2

{'v_required': 62985,
 'v_sent': 63616,
 'v_sent_full': 62901,
 'v_degraded': 715,
 'rupture_ff': 5253,
 'function_not_found': 6633,
 'v1_not_sent_from_s1': 11311,
 'v3_not_sent_from_s3': 16,
 'v_not_found_in_last_station': 84,
 'ff_required': 0,
 'ff_sent': 198260,
 'z1_VSAV_sent': 196,
 'z1_FPT_sent': 0,
 'z1_EPA_sent': 0,
 'VSAV_needed': 208,
 'FPT_needed': 0,
 'EPA_needed': 0,
 'VSAV_disp': 8,
 'FPT_disp': 5,
 'EPA_disp': 0,
 'skill_lvl': 292371.0}

In [8]:
folder_path = "./Plots"

data = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith(".pkl"):
        full_path = os.path.join(folder_path, file_name)
        with open(full_path, "rb") as f:
            d = pickle.load(f)
            name = os.path.splitext(file_name)[0]
            data[name] = d

df = pd.DataFrame.from_dict(data, orient='index')

df["skill_lvl_mean"] = df["skill_lvl"] / df["ff_sent"]
# To maximize:

df = df.rename(columns={"v_sent": "v_sent ↑", "v_sent_full": "v_sent_full ↑", "ff_sent":"ff_sent ↑"})

# To minimize:

df = df.rename(columns={"v_degraded": "v_degraded ↓", "z1_VSAV_sent": "z1_VSAV_sent ↓", "z1_FPT_sent": "z1_FPT_sent ↓", "z1_EPA_sent": "z1_EPA_sent ↓", "v1_not_sent_from_s1": "v1_not_sent_from_s1 ↓", "v3_not_sent_from_s3": "v3_not_sent_from_s3 ↓", "v_not_found_in_last_station": "v_not_found_in_ls ↓"})


df[["v_required","v_sent ↑","v_sent_full ↑", "v_degraded ↓", "v1_not_sent_from_s1 ↓", "v3_not_sent_from_s3 ↓",
    "v_not_found_in_ls ↓", "rupture_ff", 
    'z1_VSAV_sent ↓', 'z1_FPT_sent ↓', 'z1_EPA_sent ↓',
"ff_sent ↑", "skill_lvl_mean"]]

,v_required,v_sent ↑,v_sent_full ↑,v_degraded ↓,v1_not_sent_from_s1 ↓,v3_not_sent_from_s3 ↓,v_not_found_in_ls ↓,rupture_ff,z1_VSAV_sent ↓,z1_FPT_sent ↓,z1_EPA_sent ↓,ff_sent ↑,skill_lvl_mean
sim_best,62985,63531,62883,648,16331,11,102,10259,127,0,0,198557,1.134390
metrics_dt_10y_cfv1p1_rupture_ff,62985,63616,62901,715,11311,16,84,5253,196,0,0,198260,1.472112
sim_rdm_cf1_cv1_p1,62985,63616,62901,715,11311,16,84,5253,196,0,0,198260,1.474685
sim_rdm_prob,71991,72344,71490,854,24450,82,381,8747,340,0,0,224585,1.491787
sim_rdm_cf1_cv3_p1,62985,63654,62853,801,21154,67,133,6321,2931,288,27,195420,1.548470
sim_rdm_cf1_cv1_p12,75478,76330,75354,976,16139,12,124,7271,371,0,0,237360,1.481543
agent_metrics_fqf_1y_cfv1_p12_rupture_ff,75478,76449,75356,1093,14507,9,122,5385,376,0,0,237453,1.572433
sim_bst_cf1_cv1_p12,75478,76204,75304,900,24387,10,174,15300,241,0,0,237726,1.151990
sim_bst_cf1_cv3_p1,62985,63555,62820,735,27088,99,178,12535,2971,314,26,195315,1.205207
sim_bst_cf3_cv1_p1,62985,61944,51426,10518,220143,580,11559,236488,0,0,0,182683,1.874466


In [ ]:
os.chdir("./Scripts")

In [ ]:
reward_weights_folder = "Reward_weights"

reward_files = sorted([f for f in os.listdir(reward_weights_folder) if f.endswith(".json")])
reward_files

# Plots

In [ ]:
%%time

%run plot_evo.py \
agent_metrics_v_degraded_r100_cf3_reward_real.npy \
sim_metrics_r100_cf3_reward_real.npy \
--interpolation 1000

In [ ]:
%%time

%run plot_evo.py agent_metrics_v_degraded_r100_cf3_train_reward_fake.npy --interpolation 10000

In [ ]:
os.chdir("../")

# Tests

In [ ]:
os.getcwd()

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import geopandas as gpd
import random
from collections import defaultdict
from shapely.geometry import Point
from datetime import datetime, timedelta
from sklearn.neighbors import KDTree
import pickle
import argparse
from generate_environment import *

Z_1 = ['TOULOUSE - LOUGNON', 'TOULOUSE - VION']
Z_2 = ['ST JORY', 'ROUFFIAC', 'RAMONVILLE - BUCHENS', 'COLOMIERS', 'MURET - MASSAT']
Z_3 = ['AUTERIVE', 'ST LYS', 'GRENADE', 'FRONTON', 'VERFEIL', 'CARAMAN']

os.chdir('./Data')

# df_sorties = pd.read_csv("sorties_2018.csv", sep=";")
df_firestations = pd.read_csv("firestations.csv", sep = ';')
df_materiel = pd.read_csv("materiel_2018.csv", sep=";")
df_comp = pd.read_csv("comp_2018.csv", sep=";")
df_roles = pd.read_csv("roles_competences.csv", sep=";")
df_vehicles_history = pd.read_csv("df_vehicles_history.csv", sep=";")

df_xy = pd.read_csv("X-Y-lieu.csv", sep=";")
df_lieu = pd.read_csv("dbo.LIEU.csv", sep=";")
df_secteur = pd.read_csv("dbo.SECTEUR.csv", sep=";")
df_commune = pd.read_csv("dbo.COMMUNE.csv", sep=";")
df_nom_commune = pd.read_csv("dbo.NOM_COMMUNE.csv", sep=";")
df_responses = pd.read_csv("responses_by_incident.csv", sep = ';')
df_pdd = gpd.read_file("pdd.geojson")

df_stations = generate_stations(df_firestations)
stations_u = sorted(x for x in df_materiel['Nom du Centre'].unique() if not x.startswith('X') and not x.startswith('Z'))    
df_v = generate_vehicles(df_materiel, df_stations)    
df_skills, df_firefighters = generate_firefighters(df_comp)


os.chdir('../Data_environment')    

df_stations.to_pickle("df_stations.pkl")
df_v.to_pickle("df_v.pkl")
df_skills.to_pickle("df_skills.pkl")
df_roles.to_pickle("df_roles.pkl")
df_vehicles_history.to_pickle("df_vehicles_history.pkl")

os.chdir('../Data_preprocessed')

df_prob_dep = pd.read_pickle("df_prob_dep.pkl")
df_rank_incident = pd.read_pickle("df_rank_incident.pkl")

dic_inc_ar_mat = create_responses(df_responses, df_rank_incident)

# REAL

os.chdir('../Data_trained')

is_fake = False
df_pc_real = pd.read_pickle("df_real.pkl")
window = len(df_pc_real)
print("window real:", window)
df_pc_real = precompute_pdd(df_pdd, df_pc_real, stations_u)
df_pc_real = precompute_zone(df_stations, df_pc_real, Z_1, Z_2, Z_3)
df_pc_real = precompute_incident(df_rank_incident, df_pc_real)
df_pc_real = precompute_area_type(df_xy, df_lieu, df_nom_commune, df_commune, df_secteur, df_pc_real)

In [ ]:
df_pc_real = pd.concat([df_pc_real, df_prob_dep], axis=1)
prob_dict = precompute_prob_dict(df_pc_real)
print("prob_dict computed")           
df_pc_real = precompute_prob_departure(df_pc_real, prob_dict)

In [ ]:
start_year = 2018
start_inter = 1
end_inter = window

print("start_year", start_year, "start_inter", start_inter, "end_inter", end_inter)
    
df_pc_real = precompute_date(df_pc_real, start_year)
df_pc_real = precompute_returns(df_pc_real, start_inter, end_inter, is_fake)
print('real', len(df_pc_real), "done")

cols_to_norm = ["Coord X", "Coord Y", 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos']

for col in cols_to_norm:
    min_val = df_pc_real[col].min()
    max_val = df_pc_real[col].max()
    if min_val != max_val:
        df_pc_real[col] = (df_pc_real[col] - min_val) / (max_val - min_val)
    else:
        df_pc_real[col] = 0.0

os.chdir('../Data_environment')
df_pc_real.to_pickle("df_pc_real_prob.pkl")

In [ ]:
os.chdir('./Data_environment/')
df_p = pd.read_pickle("df_pc_fake_10y.pkl")
df_p

In [ ]:
os.chdir('../')
from collective_functions import *

In [ ]:
os.chdir('./Data_environment/')

In [ ]:

df_p = pd.read_pickle("df_pc_real_prob.pkl")
date_reference = df_p.date.iloc[0]
df_skills = pd.read_pickle("df_skills.pkl")
df_roles = pd.read_pickle("df_roles.pkl")
df_vehicles_history = pd.read_pickle("df_vehicles_history.pkl")
dic_roles = create_dic_roles(df_vehicles_history)
dic_roles_skills = generate_dic_roles_skills(df_roles, df_skills)
skills_updated = update_skills(df_skills, date_reference)

def gen_ff_array(df_skills, skills_updated, ff_existing):
    return skills_updated[[df_skills.index.get_loc(matricule) for matricule in ff_existing]]
ff_existing = [9122, 9123, 9124, 9125, 9126, 9131, 9132, 9133, 9134, 9135]
ff_array = gen_ff_array(df_skills, skills_updated, ff_existing)    

In [ ]:
ff_array

In [ ]:
for idx, inter in df_p.iterrows():

    num_inter, date, pdd, required_departure, zone, duration, month, day, hour, minute, \
    coord_x, coord_y, month_sin, month_cos, day_sin, day_cos, hour_sin, hour_cos = inter
    if (required_departure != {0:"RETURN"}):
        veh_depart = [v[0] for k, v in sorted(required_departure.items())]

        state = np.hstack(([get_roles_for_ff(veh, ff_array, dic_roles, dic_roles_skills) for veh in veh_depart])).astype(float)
        state /= 8 # normalization, 8 skill lvls
            
        filler = np.zeros((80-state.shape[0], state.shape[1])) # max 74 de base + 6 ff lent
        state = np.vstack((state, filler))
    
        # filler col
        try:
            filler = np.zeros((state.shape[0], 37 - state.shape[1]))
            state = np.concatenate((state, filler), axis=1)
        except:
            print(num_inter, state.shape[1])
            break

In [ ]:
df_p.iloc[8829]["departure"]

In [ ]:
prob_dict["AUTRES FEUX (SANS PRÉCISION)"]["URBAIN"]

In [ ]:
v_uniq = []

for inc, d1, in prob_dict.items():
    for area, d2 in d1.items():
        for tuple_v in d2.keys():
            for v in tuple_v:
                v_uniq.append(v)

v_uniq = sorted(set(v_uniq))

In [ ]:
v_uniq

In [ ]:
df_v_h['Type Matériel'] = df_v_h['Type Matériel'].fillna("")
df_v_h['Fonction'] = df_v_h['Fonction'].fillna("")
df_v_h['Type'] = df_v_h['Type'].fillna("")

In [ ]:
for v in v_uniq:
    if v not in df_v_h['Type Matériel'].values:
        if v not in df_v_h['Fonction'].values:
            if v not in df_v_h['Type'].values:
                print(v)

In [ ]:
df_v_h.columns

In [ ]:
comp_sort = sorted(df_comp["Compétence - Nom"].dropna().unique())

In [ ]:
li = sorted(["IMP3_CUNITE","IMP_CU", 
                   "IMP2_SAUV","IMP_SAUV", 
                   "COND_ENG_NAUT","COND_BMS_EB",
                    'CE_VLA',"*CE*",
                     'CHEF DE GROUPE',"*CDG*",
                     'CYN1_COND',"*CYN2*",
                     'CYN2_CUNITE',"*CYN3",
                     'EQ_VLA',"*EQ*",
                     'FDF_BD_EQ',"FDF_EQ",
                     'GELD_CU',
                     'GELD_EQ',
                     'INFIRMIER SP',
                     'INFIRMIER VLOSS',
                     'RAD1_EQ_RECO',
                     'RCH1_EQ_RECO',
                     'RCH2_EQ_INTER',
                     'SAL1_PLG',
                     'SAL2_PLG_CU',
                     'SAV1_SAUV',
                     'SMO1_EPIM',
                     'SMO2_EQ',
                     'SMO3_CUNITE',
                     'VLA_CE',"*CE*",
                     'VLA_EQ',"*EQ*"])

for e in li:
    if e not in comp_sort:
        print(e)

In [ ]:
comp_sort

In [ ]:
prob_dict

In [ ]:
df = pd.read_pickle("df_pc_fake_10y_p12.pkl")

In [ ]:
df["num_inter"].max()

In [ ]:
os.chdir("../")

In [ ]:
import torch
import torch.nn as nn

# Paramètres
num_pompiers = 80
num_roles = 40
embedding_dim = 40  # chaque pompier est un vecteur de 40 dim

# Exemple d'entrée
X = torch.randn(num_pompiers, num_roles)  # shape: [80, 40]

# Reshape pour MultiheadAttention
# PyTorch attend : [sequence_len, batch_size, embedding_dim]
X_input = X.unsqueeze(1)  # [80, 1, 40]

# Mécanisme d'attention
attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=4, batch_first=False)

# Appliquer l'attention
# Q = K = V = X_input (self-attention)
attn_output, attn_weights = attn(X_input, X_input, X_input)  # output: [80, 1, 40]

# Résultat final : [80, 40]
output_matrix = attn_output.squeeze(1)


In [ ]:
output_matrix

In [ ]:
from collective_functions import *
os.chdir('./Data_environment')

In [ ]:


df_stations = pd.read_pickle("df_stations.pkl")

df_v = pd.read_pickle("df_v.pkl")
dic_vehicles, dic_functions = create_dic_vehicles(df_v)
dic_vehicles = purge_dic_v(dic_vehicles)


list_of_mats = dic_vehicles["TOULOUSE - VION"]["available"]
dic_vehicles["TOULOUSE - VION"]["available"] = constrain_veh(list_of_mats, 1)

list_of_mats = dic_vehicles["TOULOUSE - LOUGNON"]["available"]
dic_vehicles["TOULOUSE - LOUGNON"]["available"] = constrain_veh(list_of_mats, 1)

df_skills = pd.read_pickle("df_skills.pkl")

df_skills = df_skills.sample(len(df_skills)//1)

df_roles = pd.read_pickle("df_roles.pkl")
dic_roles_skills = generate_dic_roles_skills(df_roles, df_skills)

df_vehicles_history = pd.read_pickle("df_vehicles_history.pkl")
dic_roles = create_dic_roles(df_vehicles_history)

In [ ]:
list(dic_functions.keys())

In [ ]:
set(e for v in dic_functions.values() for e in v)

In [ ]:
df_pc = df_pc = pd.read_pickle("df_pc_real.pkl")

In [ ]:
old_date = df_pc.iloc[0, 1]
date_reference = df_pc.iloc[0, 1]
skills_updated = update_skills(df_skills, date_reference)

In [ ]:
ff_existing = list(df_skills.index)

In [ ]:
def gen_ff_array(df_skills, skills_updated, ff_existing):
    return skills_updated[[df_skills.index.get_loc(matricule) for matricule in ff_existing]]

skup = gen_ff_array(df_skills, skills_updated, ff_existing)

In [ ]:
skup.shape

In [ ]:
unique_rows = np.unique(skup, axis=0)
nb_unique = unique_rows.shape[0]
nb_unique

In [ ]:
state = np.hstack(([get_roles_for_ff(veh, ff_array, dic_roles, dic_roles_skills) for veh in veh_depart])).astype(float)